In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
import numpy as np
from lifelines import CoxTimeVaryingFitter
from statsmodels.stats.multitest import multipletests
from collections import Counter
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

# Load metadata, relative abundance, taxonomy, and infection data

In [2]:
df_sample = pd.read_excel("../Fig3|FigS1_S3|EDFig6_9|TableS5/ST4_oralASV_alloHCT.xlsx", sheet_name='Table S4b')
df_sample['Patient ID'] = df_sample['Patient ID'].astype(str)
df_sample.columns = ['SampleID','PatientID','DayRelativeToNearestHCT','TotalLoad','OralFraction','OralLoad','GutFraction','GutLoad','Timepoint','StoolConsistency','FungalCulturability']
df_sample = df_sample.set_index('SampleID')
df_sample.head()

,PatientID,DayRelativeToNearestHCT,TotalLoad,OralFraction,OralLoad,GutFraction,GutLoad,Timepoint,StoolConsistency,FungalCulturability
SampleID,,,,,,,,,,
1000A,1000,-9,NaN,0.065440,NaN,0.934560,NaN,0,formed,NaN
1000B,1000,-4,NaN,0.270878,NaN,0.729122,NaN,5,liquid,NaN
1000C,1000,6,NaN,0.000752,NaN,0.999248,NaN,15,liquid,NaN
1000D,1000,9,NaN,0.149727,NaN,0.850273,NaN,18,semi-formed,NaN
1000E,1000,13,NaN,0.010265,NaN,0.989735,NaN,22,formed,NaN


In [3]:
df_relab = pd.read_csv("../Fig3|FigS1_S3|EDFig6_9|TableS5/relative_abundance_wide_format.csv.gz", compression="gzip", index_col=0)
df_relab.head()

,ASV_1,ASV_10,ASV_100,ASV_1000,ASV_10000,ASV_10007,ASV_10010,ASV_10011,ASV_10012,ASV_10013,...,ASV_9956,ASV_9959,ASV_997,ASV_999,ASV_9991,ASV_9992,ASV_9994,ASV_9996,ASV_9997,ASV_9998
SampleID,,,,,,,,,,,,,,,,,,,,,
574B,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1233L,0.0,0.000386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1428JJ,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1532B,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1814Z,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_tax = pd.read_csv("../Fig3|FigS1_S3|EDFig6_9|TableS5/taxonomy.csv")
df_tax.head()

,ASV,Sequence,Kingdom,Phylum,Class,Order,Family,Genus,LowestClassifiedTaxa,ClosestHMPTaxa,ConfidenceKingdom,ConfidencePhylum,ConfidenceClass,ConfidenceOrder,ConfidenceFamily,ConfidenceGenus,TaxonomyColor,TaxonomyColorOrder
0,ASV_1,AGCGCAGGCGGTTGCTTAGGTCTGATGTGAAAGCCTTCGGCTTAAC...,Bacteria,Firmicutes,Bacilli,Lactobacillales,Lactobacillaceae,Lactobacillus,Lactobacillus,Lactobacillus,100,100,100,100,100,100,#1635A4,163
1,ASV_10,AGCGTAGACGGAAGAGCAAGTCTGATGTGAAAGGCTGGGGCTTAAC...,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Blautia,Blautia,Blautia,100,100,100,100,100,93,#AD998C,7
2,ASV_100,AGCGTAGATGGATGTTTAAGTCAGTTGTGAAAGTTTGCGGCTCAAC...,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides,Bacteroides,100,100,100,100,100,100,#16DDD3,44
3,ASV_1000,TGCGTAGGTGGTTTCTTAAGTCAGAGGTGAAAGGCTACGGCTCAAC...,Bacteria,Firmicutes,Clostridia,Peptostreptococcales-Tissierellales,Peptostreptococcaceae,Romboutsia,Romboutsia,Clostridia,100,100,100,100,100,100,#BEA89A,21
4,ASV_10000,AGCGTAGACGGTGTGGCAAGTCTGATGTGAAAGGCATGGGCTCAAC...,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Blautia,Blautia,Blautia,100,100,100,100,100,100,#BEA89A,21


In [5]:
df_inf = pd.read_csv('tblInfectionsCidPapers_add_Strep.csv')
df_inf = df_inf[df_inf.DayRelativeToNearestHCT.notnull()] # remove patients with unknown HCT day
df_inf.head()

,PatientID,Timepoint,InfectiousAgent,DayRelativeToNearestHCT
0,1000,213,Enterococcus_Faecium,204.0
1,1003,1046,Enterococcus_Faecium_Vancomycin_Resistant,1049.0
2,1010,-58,Escherichia,-61.0
3,1015,19,Enterococcus_Faecium_Vancomycin_Resistant,16.0
4,1015,20,Enterococcus_Faecium_Vancomycin_Resistant,17.0


In [6]:
set(df_inf.InfectiousAgent)

{'Citrobacter',
 'Enterobacter',
 'Enterococcus_Faecalis',
 'Enterococcus_Faecium',
 'Enterococcus_Faecium_Vancomycin_Resistant',
 'Enterococcus_Vancomycin_Resistant',
 'Escherichia',
 'Klebsiella',
 'Klebsiella_Pneumoniae',
 'Pseudomonas',
 'Stenotrophomonas_Maltophilia',
 'Streptococcus_Agalactiae',
 'Streptococcus_Anginosus',
 'Streptococcus_Oralis',
 'Streptococcus_Pneumoniae',
 'Streptococcus_Viridans_Group'}

In [9]:
df_oralasv = pd.read_excel("../Fig3|FigS1_S3|EDFig6_9|TableS5/ST4_oralASV_alloHCT.xlsx", sheet_name="Table S4a", index_col=0)
df_oralasv.head()

,Sequence,Kingdom,Phylum,Class,Order,Family,Genus
ASV,,,,,,,
ASV_10020,AGCGCAGGCGGGCATGTAAGTCTTTCTTAAAAGTTCGGGGCTCAAC...,Bacteria,Firmicutes,Negativicutes,Veillonellales-Selenomonadales,Selenomonadaceae,Centipeda
ASV_10022,CATCTAGGCGGCCAGATAAGTCCGAGGTGAAAACTGCCGGCTCAAC...,Bacteria,Fusobacteriota,Fusobacteriia,Fusobacteriales,Leptotrichiaceae,Leptotrichia
ASV_10082,TGCGTAGGTGGCGTATTAAGTCAGTGGTGAAAAGCTGCAGCTCAAC...,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Porphyromonadaceae,Porphyromonas
ASV_10084,AGCGTAGGCGGATTATTAAGTCAGTGGTGAAAGACGGTGGCTCAAC...,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Prevotellaceae,Alloprevotella
ASV_10143,CATCTAGGCGGCCAGACAAGTCTGGGGTGAAAACTTGCGGCTCAAC...,Bacteria,Fusobacteriota,Fusobacteriia,Fusobacteriales,Leptotrichiaceae,Leptotrichia


# Build covariate table

In [10]:
df_cov = None
for genus in ['Klebsiella','Enterococcus','Streptococcus','Escherichia-Shigella','Oral']:
    if genus == 'Oral':
        genus_asvs = list(df_oralasv.index)
    else:
        genus_asvs = list(df_tax[df_tax.Genus==genus].ASV)
    df_tmp = (df_relab[set(df_relab.columns).intersection(set(genus_asvs))]>0.3).astype(int)
    df_tmp = (df_tmp.sum(axis=1) > 0).astype(int).to_frame()
    df_tmp.columns = [genus]
    if df_cov is None:
        df_cov = deepcopy(df_tmp)
    else:
        df_cov = pd.merge(df_cov, df_tmp, left_index=True, right_index=True, how='inner')
df_cov = pd.merge(df_sample[['PatientID','Timepoint','DayRelativeToNearestHCT']], df_cov, left_index=True, right_index=True, how='inner')
df_cov.head()

,PatientID,Timepoint,DayRelativeToNearestHCT,Klebsiella,Enterococcus,Streptococcus,Escherichia-Shigella,Oral
SampleID,,,,,,,,
1000A,1000,0,-9,0,0,0,0,0
1000B,1000,5,-4,0,0,0,0,0
1000C,1000,15,6,0,0,0,0,0
1000D,1000,18,9,0,0,0,0,0
1000E,1000,22,13,0,0,0,0,0


# Count number of dominations

In [11]:
df2_cov = deepcopy(df_cov)
df2_cov = df2_cov[(df2_cov.DayRelativeToNearestHCT >= -10) & (df2_cov.DayRelativeToNearestHCT <= 40)]
pids_to_keep = []
for key, value in dict(Counter(df2_cov.PatientID)).items():
    if value >= 5:
        pids_to_keep.append(key)
df2_cov = df2_cov[df2_cov.PatientID.isin(pids_to_keep)]

for genus in ['Klebsiella','Enterococcus','Streptococcus','Escherichia-Shigella','Oral']:
    df3_cov = df2_cov[df2_cov[genus]==1]
    print(genus, len(set(df3_cov.PatientID)))

Klebsiella 38
Enterococcus 280
Streptococcus 221
Escherichia-Shigella 50
Oral 234


# Build Cox model

In [21]:
def build_cox_model_input_table(
    covariates,
    infectious_agent,
    study_start_day=-10,
    study_stop_day=40,
    min_samples_per_patient=5,
    min_domination_times=10,
    memory_duration = np.inf
):    
    lines = []
    for pid in set(df_sample.PatientID):
        df_sample_pid = df_sample[(df_sample.PatientID==pid)&(df_sample.DayRelativeToNearestHCT>=study_start_day)&(df_sample.DayRelativeToNearestHCT<=study_stop_day)].sort_values('Timepoint')
        if len(df_sample_pid) == 0:
            continue
        
        # check if the patient receives multiple HCT
        # treat each transplant as individual
        tps = list(df_sample_pid.Timepoint)
        days = list(df_sample_pid.DayRelativeToNearestHCT)
        break_tps = [tps[0]-1]
        for xt, yt, xd, yd in zip(tps, tps[1:], days, days[1:]):
            if yt-xt != yd-xd:
                break_tps.append(xt)
        break_tps.append(tps[-1])

        # break_tps = [tp1, tp2, tp3, ...]
        # periods include (tp1, tp2], (tp2, tp3], ...
        for tps_from, tps_to in zip(break_tps, break_tps[1:]):
            df_sample_tmp = df_sample_pid[(df_sample_pid.Timepoint>tps_from) & (df_sample_pid.Timepoint<=tps_to)]

            # skip if the current transplant period has fewer than min_samples_per_patient samples
            if len(df_sample_tmp)<min_samples_per_patient:
                continue
            else:
                first_sample_day = int(list(df_sample_tmp.DayRelativeToNearestHCT)[0])
                last_sample_day = int(list(df_sample_tmp.DayRelativeToNearestHCT)[-1])
                #print(first_sample_day, last_sample_day)
                assert first_sample_day < last_sample_day

            # find first infection (use timepoint to filter)
            df_inf_tmp = df_inf[(df_inf.PatientID==pid) & (df_inf.InfectiousAgent.str.contains('|'.join(infectious_agent))) & (df_inf.Timepoint>tps_from) & (df_inf.Timepoint<=tps_to)]
            df_inf_tmp = df_inf_tmp[(df_inf_tmp.DayRelativeToNearestHCT>first_sample_day) & (df_inf_tmp.DayRelativeToNearestHCT<=last_sample_day)]
            if len(df_inf_tmp) == 0:
                # no infection detected, set first infection day as the day of the last sample (censoring time)
                first_infection_day = last_sample_day
                infection_event = 0
            else:
                infection_days = list(df_inf_tmp.DayRelativeToNearestHCT)
                infection_days = [day for day in infection_days if day > first_sample_day]
                if len(infection_days) == 0:
                    first_infection_day = last_sample_day
                    infection_event = 0
                else:
                    first_infection_day = infection_days[0]
                    infection_event = 1
            assert first_infection_day > first_sample_day
            
            # if pid=='634':
            #     print(first_infection_day)
            #     assert 0
            
            # use intestinal domination as covariant
            curr_df_cov = df_cov[(df_cov.PatientID==pid) & (df_cov.Timepoint>tps_from) & (df_cov.Timepoint<=tps_to)]
            if len(curr_df_cov) >0:

                # reformat by giving each row a particular day and each column a dominating genus
                curr_df_cov = curr_df_cov.drop(['PatientID','Timepoint'], axis=1).sort_values('DayRelativeToNearestHCT').set_index('DayRelativeToNearestHCT', drop=True)
                curr_df_cov = curr_df_cov[covariates]

                # add covariates and event status for each period
                curr_state = [0] * len(covariates)
                # use drug exposure on the first sample day as the initial states
                if first_sample_day in list(curr_df_cov.index):
                    prev_state = list(curr_df_cov.loc[first_sample_day])
                else:
                    prev_state = [0] * len(covariates)
                days_after_measurement = [0] * len(covariates)
                for day in np.arange(first_sample_day, first_infection_day):
                    # for each period (left, right]: left is exclusive, right is inclusive
                    left_day = day    # exclusive
                    right_day = day+1 # inclusive

                    # get right day covariates
                    # domination_records are antibiotic exposure on the right day
                    if right_day in list(curr_df_cov.index):
                        domination_records = list(curr_df_cov.loc[right_day])
                    else:
                        domination_records = [0] * len(covariates)

                    # get current drug impact states
                    # since we consider antibiotic pharmacokinetics, antibiotic may impact microbiome over several days
                    curr_state = deepcopy(domination_records)
                    for k,r in enumerate(domination_records):
                        if domination_records[k]==1:
                            curr_state[k]=1
                            days_after_measurement[k] = 0 # reset the clock
                        else:
                            if days_after_measurement[k]>=memory_duration:
                                curr_state[k] = 0
                            else:
                                curr_state[k] = deepcopy(prev_state)[k]
                            days_after_measurement[k] += 1

                    # combine with previous records
                    curr_line = [pid, left_day, right_day]
                    if (day == first_sample_day) or (curr_state != prev_state):
                        curr_line.extend(curr_state)
                        if right_day == first_infection_day:
                            curr_line.append(infection_event)
                        else:
                            curr_line.append(0)
                        lines.append(curr_line)
                        prev_state = deepcopy(curr_state) # must be deepcopy! otherwise, change in curr_state will lead to the same change to prev_state
                    else:
                        # modify last entry in lines
                        last_line = lines[-1]
                        last_line[2] = right_day
                        if right_day == first_infection_day:
                            last_line[-1] = infection_event
                        else:
                            last_line[-1] = 0
                        lines[-1] = last_line                        
            else:
                lines.append([pid, first_sample_day, first_infection_day] + [0]*len(covariates) + [infection_event])
    
    df_cox = pd.DataFrame(lines, columns=['PatientID','StartDay','StopDay']+covariates+['Infection'])
    
    # remove drugs that have been used for less than 10 times
    dominatinggenus2drop = []
    for genus in df_cox.iloc[:,3:-1]:
        if np.sum(df_cox[genus]) < min_domination_times:
            dominatinggenus2drop.append(genus)
    df_cox = df_cox.drop(dominatinggenus2drop, axis=1)
        
    return df_cox

# Oral ASV ~ total BSI

In [33]:
# build input table
df_cox_wo_mem = build_cox_model_input_table(
    covariates=['Oral'],
    infectious_agent=list(set(df_inf.InfectiousAgent))
)

# run cox regression at panelty=0
ctv_wo_mem = CoxTimeVaryingFitter(penalizer=0)
ctv_wo_mem.fit(df_cox_wo_mem, id_col="PatientID", event_col="Infection", start_col="StartDay", stop_col="StopDay", show_progress=False, fit_options={'step_size':0.5})
df_cox_summary = ctv_wo_mem.summary[['exp(coef)','exp(coef) lower 95%','exp(coef) upper 95%','p']]

# remove drugs with insufficient data to support its correlation
df_cox_summary = df_cox_summary.sort_values('exp(coef)')
df_cox_summary['padj'] = multipletests(df_cox_summary['p'], method='fdr_bh')[1]
df_cox_summary

,exp(coef),exp(coef) lower 95%,exp(coef) upper 95%,p,padj
covariate,,,,,
Oral,0.254926,0.110257,0.589417,0.001393,0.001393


In [35]:
len(df_cox_wo_mem[df_cox_wo_mem.Infection==1])

86

# Escherichia-Shigella ~ Escherichia BSI

In [22]:
# build input table
df_cox_wo_mem = build_cox_model_input_table(
    covariates=['Klebsiella','Enterococcus','Streptococcus','Escherichia-Shigella'],
    infectious_agent=['Escherichia']
)

# run cox regression at panelty=0
ctv_wo_mem = CoxTimeVaryingFitter(penalizer=0)
ctv_wo_mem.fit(df_cox_wo_mem, id_col="PatientID", event_col="Infection", start_col="StartDay", stop_col="StopDay", show_progress=False, fit_options={'step_size':0.5})
df_cox_summary = ctv_wo_mem.summary[['exp(coef)','exp(coef) lower 95%','exp(coef) upper 95%','p']]

# remove drugs with insufficient data to support its correlation
df_cox_summary = df_cox_summary.sort_values('exp(coef)')
df_cox_summary['padj'] = multipletests(df_cox_summary['p'], method='fdr_bh')[1]
df_cox_summary

,exp(coef),exp(coef) lower 95%,exp(coef) upper 95%,p,padj
covariate,,,,,
Klebsiella,5.554798e-08,0.000000,inf,9.954264e-01,9.954264e-01
Streptococcus,1.544588e-01,0.020915,1.140690,6.711087e-02,1.342217e-01
Enterococcus,1.641828e+00,0.724969,3.718226,2.345172e-01,3.126896e-01
Escherichia-Shigella,1.562858e+01,7.547467,32.362187,1.337287e-13,5.349147e-13


In [23]:
len(df_cox_wo_mem[df_cox_wo_mem.Infection==1])

32

# Enterococcus ~ Enterococcus BSI

In [24]:
# build input table
df_cox_wo_mem = build_cox_model_input_table(
    covariates=['Klebsiella','Enterococcus','Streptococcus','Escherichia-Shigella'],
    infectious_agent=['Enterococcus']
)

# run cox regression at panelty=0
ctv_wo_mem = CoxTimeVaryingFitter(penalizer=0)
ctv_wo_mem.fit(df_cox_wo_mem, id_col="PatientID", event_col="Infection", start_col="StartDay", stop_col="StopDay", show_progress=False)
df_cox_summary = ctv_wo_mem.summary[['exp(coef)','exp(coef) lower 95%','exp(coef) upper 95%','p']]

# remove drugs with insufficient data to support its correlation
df_cox_summary = df_cox_summary.sort_values('exp(coef)')
df_cox_summary['padj'] = multipletests(df_cox_summary['p'], method='fdr_bh')[1]
df_cox_summary

,exp(coef),exp(coef) lower 95%,exp(coef) upper 95%,p,padj
covariate,,,,,
Klebsiella,6.650986e-08,0.000000,inf,9.952957e-01,9.952957e-01
Streptococcus,3.283696e-01,0.077604,1.389439,1.302585e-01,2.605171e-01
Escherichia-Shigella,5.093103e-01,0.069493,3.732692,5.067514e-01,6.756685e-01
Enterococcus,8.362555e+00,3.743879,18.679111,2.224248e-07,8.896992e-07


In [25]:
len(df_cox_wo_mem[df_cox_wo_mem.Infection==1])

36

# Klebsiella ~ Klebsiella BSI

In [26]:
# build input table
df_cox_wo_mem = build_cox_model_input_table(
    covariates=['Klebsiella','Enterococcus','Streptococcus','Escherichia-Shigella'],
    infectious_agent=['Klebsiella']
)

# run cox regression at panelty=0
ctv_wo_mem = CoxTimeVaryingFitter(penalizer=0)
ctv_wo_mem.fit(df_cox_wo_mem, id_col="PatientID", event_col="Infection", start_col="StartDay", stop_col="StopDay", show_progress=False)
df_cox_summary = ctv_wo_mem.summary[['exp(coef)','exp(coef) lower 95%','exp(coef) upper 95%','p']]

# remove drugs with insufficient data to support its correlation
df_cox_summary = df_cox_summary.sort_values('exp(coef)')
df_cox_summary['padj'] = multipletests(df_cox_summary['p'], method='fdr_bh')[1]
df_cox_summary

,exp(coef),exp(coef) lower 95%,exp(coef) upper 95%,p,padj
covariate,,,,,
Streptococcus,0.170125,0.018805,1.539054,0.114967,0.229933
Enterococcus,0.340497,0.067473,1.718284,0.192062,0.256083
Escherichia-Shigella,2.437807,0.515286,11.533221,0.261101,0.261101
Klebsiella,13.672557,2.707619,69.041771,0.001548,0.006191


In [27]:
len(df_cox_wo_mem[df_cox_wo_mem.Infection==1])

13

# Streptococcus ~ Streptococcus BSI

In [28]:
# build input table
df_cox_wo_mem = build_cox_model_input_table(
    covariates=['Klebsiella','Enterococcus','Streptococcus','Escherichia-Shigella'],
    infectious_agent=['Streptococcus']
)

# run cox regression at panelty=0
ctv_wo_mem = CoxTimeVaryingFitter(penalizer=0)
ctv_wo_mem.fit(df_cox_wo_mem, id_col="PatientID", event_col="Infection", start_col="StartDay", stop_col="StopDay", show_progress=False)
df_cox_summary = ctv_wo_mem.summary[['exp(coef)','exp(coef) lower 95%','exp(coef) upper 95%','p']]

# remove drugs with insufficient data to support its correlation
df_cox_summary = df_cox_summary.sort_values('exp(coef)')
df_cox_summary['padj'] = multipletests(df_cox_summary['p'], method='fdr_bh')[1]


df_cox_summary

,exp(coef),exp(coef) lower 95%,exp(coef) upper 95%,p,padj
covariate,,,,,
Enterococcus,8.695999e-08,0.0,inf,0.996251,0.999031
Streptococcus,9.599968e-08,0.0,inf,0.996717,0.999031
Escherichia-Shigella,1.423054e-07,0.0,inf,0.997992,0.999031
Klebsiella,4.205856e-07,0.0,inf,0.999031,0.999031


In [29]:
len(df_cox_wo_mem[df_cox_wo_mem.Infection==1])

1